In [1]:
import sys
from copy import deepcopy

def distance(loc1, loc2):
    return ((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)**0.5

In [2]:
class RideSharing:

    requests = []
    tours = []
    epochs = []

    def add_request(self, pickup_loc, dropoff_loc, request_time, max_waiting_duration):
        self.requests.append(Request(pickup_loc, dropoff_loc, request_time, max_waiting_duration))

    def remove_request(self, request):
        if request in self.requests:
            self.requests.remove(request)

    def add_tour(self, depot_loc, latest_return=sys.maxsize):
        self.tours.append(Tour(depot_loc, latest_return))

    def remove_tour(self, tour):
        if tour in self.tours:
            self.tours.remove(tour)

    def solve(self):
        self.requests = sorted(self.requests, key=lambda x: x.request_time)
        for request in self.requests:
            new_epoch = DecisionEpoch(request, self.tours, self.beta, self.gamma1, self.gamma2)
            self.tours = new_epoch.tours
            self.epochs.append(new_epoch)
        
    def __init__(self, beta, gamma1, gamma2):
        self.beta = beta
        self.gamma1 = gamma1
        self.gamma2 = gamma2

In [3]:
class Location:

    def __init__(self, loc, request):
        self.loc = loc
        self.request = request


class PickUp(Location):
    is_pickup = True
    is_dropoff = False

    def __init__(self, loc, request):
        super().__init__(loc, request)


class DropOff(Location):
    is_pickup = False
    is_dropoff = True

    def __init__(self, loc, request):
        super().__init__(loc, request)

In [4]:
class Request:

    count = 0

    def __init__(self, pickup_loc, dropoff_loc, request_time, max_waiting_duration):
        self.id = Request.count
        Request.count += 1
        self.request_time = request_time
        self.max_waiting_duration = max_waiting_duration
        self.latest_dropoff = request_time + distance(pickup_loc, dropoff_loc) + max_waiting_duration
        self.pickup = PickUp(pickup_loc, self)
        self.dropoff = DropOff(dropoff_loc, self)

In [5]:
class Tour:

    def __init__(self, depot_loc=(0, 0), M=sys.maxsize):
        depot = Request(depot_loc, depot_loc, 0, M)
        self.tour = [depot.pickup, depot.dropoff]
        self.early = [0, 0]
        self.late = [M, M]
        self.length = 2
        self.total_cost = 0

    def feasible(self, place, i):
        if place.is_dropoff and place.request.pickup not in self.tour[:i]:
            return False
        if place.is_pickup and place.request.dropoff in self.tour[:i]:
            return False
        e, l = self.insertion_time_window(place, i)
        return e <= l

    def insertion_cost(self, place, i):
        return (distance(self.tour[i-1].loc, place.loc) + distance(place.loc, self.tour[i].loc)
                - distance(self.tour[i-1].loc, self.tour[i].loc))

    # berechne Zeitfenster
    def insertion_time_window(self, place, i):
        # e alt:
        # e = max(place.request.req_t, self.early[i-1] + euclidean(self.tour[i-1].loc, place.loc))
        # e neu:
        e = max(place.request.request_time + distance(self.tour[i-1].loc, place.loc), 
                self.early[i-1] + distance(self.tour[i-1].loc, place.loc))
        l = min(place.request.latest_dropoff, 
                self.late[i] - distance(place.loc, self.tour[i].loc))
        return e, l

    def insert(self, place, i):
        self.total_cost += self.insertion_cost(place, i)
        self.tour.insert(i, place)
        self.length += 1
        self.update(i)
        
    def remove(self, i):
        place = self.tour[i]
        del self.tour[i]
        del self.early[i]
        del self.late[i]
        self.total_cost -= self.insertion_cost(place, i)
        self.length -= 1
        self.recalculate_time_windows()

    def update(self, i):
        # Berechnung und Einfügen des Zeitfensters des neuen Ortes
        e, l = self.insertion_time_window(self.tour[i], i)
        self.early.insert(i, e)
        self.late.insert(i, l)
        # Update der Zeitfenster (late) der vorigen Orte
        for k in range(i-1, -1, -1):
            self.late[k] = min(self.late[k], self.late[k+1] - distance(self.tour[k].loc, self.tour[k+1].loc))
        # Update der Zeitfenster (early) der nachfolgenden Orte
        for k in range(i+1, self.length):
            self.early[k] = max(self.early[k], self.early[k-1] + distance(self.tour[k-1].loc, self.tour[k].loc))
    
    def recalculate_time_windows(self):
        for k in range(1, self.length):
            self.early[k] = max(self.tour[k].request.request_time + distance(self.tour[k-1].loc, self.tour[k].loc), 
                self.early[k-1] + distance(self.tour[k-1].loc, self.tour[k].loc))
        for k in range(self.length-2, -1, -1):
            self.late[k] = min(self.tour[k].request.latest_dropoff, 
                self.late[k+1] - distance(self.tour[k].loc, self.tour[k+1].loc))
        
    def printable(self):
        result = []
        for i in range(self.length):
            if i == 0 or i == self.length-1:
                typ = "Depot"
            elif self.tour[i].is_pickup:
                typ = "Pick-Up"
            else:
                typ = "Drop_Off"
            result.append((self.tour[i].request.id, 
                           self.tour[i].loc, 
                           round(self.early[i], 2), 
                           round(self.late[i], 2), 
                           typ))
        return result

In [6]:
class DecisionEpoch:
    
    # Customers in a vehicle at new request time
    in_vehicle = []
    
    # Customers waiting for a vehicle at new request time
    waiting = []
    
    def __init__(self, new_request, tours, beta, gamma1, gamma2):
        self.time = new_request.request_time
        self.request = new_request
        self.tours = deepcopy(tours)
        for i in range(len(tours)): # Touren
            v = len([e for e in self.tours[i].early if e <= self.time]) # Trenne Besuchszeiten auf: Vor und nach Eingang
            for j in range(v):
                if tours[i].tour[j].is_pickup:
                    for k in range(v, tours[i].length):
                        if tours[i].tour[v].request.dropoff == tours[i].tour[k]:
                            self.in_vehicle.append((i, j, k))
            for j in range(v, tours[i].length):
                if tours[i].tour[j].is_pickup:
                    for k in range(i+1, tours[i].length):
                        if tours[i].tour[v].request.dropoff == tours[i].tour[k]:
                            self.waiting.append((i, j, k))
        self.accept, self.solution = LNS(self, "accept", beta, gamma1, gamma2)
        if self.accept:
            self.tours = self.solution
            self.accept, self.solution = LNS(self, "re-routing", beta, gamma1, gamma2)

In [7]:
def parallel_insertion(tours, requests, time=0, M=sys.maxsize):
    reject = []
    # Kunden = copy.deepcopy(fiktivekunden)
    
    while requests:
        pOpt = M 
        for j in range(len(requests)):
            for k in range(len(tours)):
                # + 1?
                earl_pos = min(len([e for e in tours[k].early if e <= time])+1, tours[k].length-1)
                # spos: Position in Tour k, an der der Pickup des Kunden eingefügt wird
                for spos in range(earl_pos, tours[k].length):
                    if tours[k].feasible(requests[j].pickup, spos):
                        skosten = tours[k].insertion_cost(requests[j].pickup, spos)
                        
                        # speichert die aktuell gültige Tour in einer temporären Variablen
                        tour_temp = deepcopy(tours[k])
                        
                        # füge den Pickup des Kunden vorübergehend in den Tourenplan ein und Update der Zeitfenster
                        tour_temp.insert(requests[j].pickup, spos)
                        
                        # Prüfe für alle Positionen im Tourenplan nach dem eingefügten Pickup ob es einen möglichen Drop-off gibt 
                        for epos in range(spos+1, tour_temp.length): 
                            if tour_temp.feasible(requests[j].dropoff, epos):  
                                ekosten = tour_temp.insertion_cost(requests[j].dropoff, epos)
                                if skosten + ekosten < pOpt:
                                    reqOpt = requests[j] 
                                    tourOpt = tours[k] 
                                    sposOpt = spos
                                    eposOpt = epos
                                    pOpt = skosten + ekosten   
      
        if pOpt < M:   
            # Setze den Pickup und Drop-Off des Kunden ein und aktualisiere Zeitfenster
            tourOpt.insert(reqOpt.pickup, sposOpt)
            tourOpt.insert(reqOpt.dropoff, eposOpt)
            
            #Entferne den eigesetzten Kunden aus der Liste der noch ausstehenden Kunden
            requests.remove(reqOpt)         
        else:
            # Wenn kein Kunde den Feasibility mehr besteht, speichere die abgewiesenen Kunden und leere die Liste
            reject = deepcopy(requests)
            requests = []
   
    return tours, reject

In [8]:
from random import sample, randint
from math import floor, ceil

def random_removal(requests, gamma1, gamma2):
    q = len(requests)
    n = randint(floor(gamma1 * q), ceil(gamma2 * q))
    return sample(requests, n)
    
def worst_removal():
    pass

def shaw_removal():
    pass

def LNS(epoch, type, beta, gamma1, gamma2):
    plan_current = deepcopy(epoch.tours)
    plan_best = deepcopy(epoch.tours)
    unplanned = [epoch.request]
    # Tiefe Kopie nicht vergessen
    removable = epoch.waiting
    all_insertable = False
    iteration = 0
    while iteration < beta:
        iteration += 1
        plan_new = deepcopy(plan_current)
        to_remove = random_removal(removable, gamma1, gamma2)
        temp = [[] for i in range(len(epoch.tours))]
        for elem in to_remove:
            unplanned.append(elem[1].request)
            temp[elem[0]].append(elem[1])
            temp[elem[0]].append(elem[2])
        for i in range(len(temp)):
            temp[i].sort(reverse=True)
            for pos in temp[i]:
                plan_new[i].remove(pos)
        plan_new, reject = parallel_insertion(plan_new, unplanned, epoch.time)
        if len(reject) == 0 and type=="accept":
            plan_best = plan_new
            return True, plan_best
        elif len(reject) == 1:
            if epoch.request not in reject:
                continue
            plan_current = plan_new
            length_current = sum([tour.length for tour in plan_current])
            length_best = sum([tour.length for tour in plan_best])
            # Kosten: Beste Restreisezeit nicht komplette Reisezeit
            cost_current = sum([tour.total_cost for tour in plan_current])
            cost_best = sum([tour.total_cost for tour in plan_best])
            if length_current > length_best:
                all_insertable = True
                plan_best = plan_current
                removable.append(epoch.request)
                unplanned = []
            elif cost_current < cost_best:
                plan_best = plan_current
    return all_insertable, plan_best

In [10]:
instance = RideSharing(10, 0.2, 0.8)
instance.add_tour((1, 1))
instance.add_tour((6, 6))
instance.add_request((1, 8), (5, 9), 1, 10)
instance.add_request((6, 5), (3, 4), 0, 10)
instance.add_request((3, 2), (8, 5), 0, 10)
instance.solve()

TypeError: 'Request' object does not support indexing

In [ ]:
for tour in instance.tours:
    print(tour.printable())
    print(tour.length)